In [5]:
from ragatouille import RAGPretrainedModel

In [6]:
index = RAGPretrainedModel.from_index(".ragatouille/colbert/indexes/fever-articles-10-index")

[Oct 14, 14:58:29] Loading segmented_maxsim_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...


/Users/siva/pz/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/siva/pz/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/Users/siva/pz/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [9]:
out = index.search("Jab Tak Hai Jaan was praised for its chemistry between its lead singers.", k=5)

In [12]:
out[0]['content']

"She finds the diary of Samar Anand -LRB- Shah Rukh Khan -RRB- , a bomb-disposal expert . The diary recounts his time as a struggling immigrant in London , and later details his whirlwind romance with Meera Thapar -LRB- Katrina Kaif -RRB- . The film was released during the six-day Diwali weekend beginning on 13 November 2012 . Jab Tak Hai Jaan emerged as the third-highest-grossing Bollywood film overseas at that time , after 3 Idiots and My Name Is Khan -LRB- 2010 -RRB- .   The film was praised for its direction , cinematography , and the chemistry between its lead actors . Khan and Sharma were lauded for their performances , though the movie 's predictable plot drew criticism . At the end of its theatrical run Jab Tak Hai Jaan received a number of awards , including four Filmfare Awards , which included trophies for Best Supporting Actress -LRB- Anushka Sharma -RRB- and Best Lyrics for Gulzar ."

In [13]:
# Accuracy of FEVER

In [ ]:
def accuracy():
    # read record json
    
    
    # read ground truth file
    # reindex the ground truth to map from <id> to label: <true/false>
    
    